In [1]:
Train_path="D:\\AI\\Data\\Malaria Cell\\cell_images\\cell_images\\Train"
Test_path="D:\\AI\\Data\\Malaria Cell\\cell_images\\cell_images\\Test"

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
import cv2

In [4]:
input_image=cv2.imread("D:\\AI\\Data\\Malaria Cell\\cell_images\\cell_images\\Train\\Parasitized\\C33P1thinF_IMG_20150619_114756a_cell_179.png")

In [5]:
input_image.shape

(163, 142, 3)

In [6]:
training_datagen=ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2)

In [7]:
training_dataset=training_datagen.flow_from_directory(Train_path,
                                                      target_size=(150,150),
                                                      batch_size=32,
                                                      class_mode="binary"
                                                     )

Found 27558 images belonging to 2 classes.


In [8]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [9]:
test_dataset=test_datagen.flow_from_directory(Test_path,
                                             target_size=(150,150),
                                             batch_size=32,
                                             class_mode="binary")

Found 27558 images belonging to 2 classes.


In [10]:
training_dataset.class_indices

{'Parasitized': 0, 'Uninfected': 1}

In [11]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten

In [12]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation ="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(2,activation="softmax"))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 16)      208       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 32)        2080      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 32)        4128      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 32)        0

In [14]:
from tensorflow.keras.optimizers import Adam

In [15]:
model.compile(optimizer=Adam(lr=0.0001),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [16]:
history=model.fit(training_dataset,epochs=3,validation_data=test_dataset,verbose=1)

Epoch 1/3
862/862 [==============================] - 684s 792ms/step - loss: 0.6944 - accuracy: 0.5417 - val_loss: 0.6614 - val_accuracy: 0.6048
Epoch 2/3
862/862 [==============================] - 680s 789ms/step - loss: 0.6372 - accuracy: 0.6470 - val_loss: 0.6640 - val_accuracy: 0.5489
Epoch 3/3
862/862 [==============================] - 691s 802ms/step - loss: 0.6304 - accuracy: 0.6550 - val_loss: 0.6663 - val_accuracy: 0.5159


In [17]:
model.save("malaria_model.h5")